# 🛸 Quantum Route Optimization for PranAIR Medical Drone

This notebook demonstrates the implementation of **Quantum-Inspired Route Optimization** using **QUBO (Quadratic Unconstrained Binary Optimization)** formulation to solve the Traveling Salesman Problem (TSP) for medical emergency drone delivery.

## 🎯 Objectives
1. Implement QUBO-based TSP solver using Qiskit
2. Optimize medical emergency delivery routes
3. Integrate with FastAPI backend
4. Compare classical vs quantum approaches
5. Visualize optimized routes

## ⚛️ Technology Stack
- **Qiskit**: Quantum computing framework
- **NetworkX**: Graph algorithms
- **NumPy**: Numerical computation
- **FastAPI**: Backend integration
- **Matplotlib**: Visualization

---

## 1️⃣ Import Required Libraries

In [ ]:
# Standard Libraries
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import time
import json
from typing import List, Tuple, Optional

# Quantum Optimization Libraries
try:
    from qiskit_optimization.applications import Tsp
    from qiskit_optimization.converters import QuadraticProgramToQubo
    from qiskit_algorithms import NumPyMinimumEigensolver
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    OPTIMIZER_AVAILABLE = True
    print("✅ Qiskit Optimization libraries loaded successfully")
except ImportError as e:
    OPTIMIZER_AVAILABLE = False
    print(f"❌ Qiskit libraries not available: {e}")
    print("Install with: pip install qiskit qiskit-optimization qiskit-algorithms")

# Import our custom optimizer
try:
    import sys
    sys.path.append('.')
    from quantum_route_optimizer import (
        solve_tsp_qubo,
        calculate_route_metrics,
        haversine_distance,
        Location
    )
    print("✅ Custom quantum optimizer imported")
except ImportError as e:
    print(f"⚠️ Could not import custom optimizer: {e}")

print(f"\n🎯 Optimizer Status: {'READY' if OPTIMIZER_AVAILABLE else 'NOT AVAILABLE'}")

## 2️⃣ Define Emergency Locations (Delhi NCR)

In [ ]:
# Define Drone Base and Emergency Locations
drone_base = Location(lat=28.6139, lng=77.2090, id="Drone_Base_Connaught_Place")

emergency_locations = [
    Location(lat=28.6280, lng=77.2207, id="Emergency_1_AIIMS"),
    Location(lat=28.5494, lng=77.2501, id="Emergency_2_Nehru_Place"),
    Location(lat=28.6692, lng=77.4538, id="Emergency_3_Noida_Hospital"),
    Location(lat=28.4595, lng=77.0266, id="Emergency_4_Gurgaon_Sector"),
    Location(lat=28.7041, lng=77.1025, id="Emergency_5_Delhi_University")
]

print("🚁 Drone Base Location:")
print(f"   {drone_base.id}: ({drone_base.lat}, {drone_base.lng})\n")

print("🚨 Emergency Delivery Targets:")
for i, loc in enumerate(emergency_locations, 1):
    print(f"   {i}. {loc.id}: ({loc.lat}, {loc.lng})")

## 3️⃣ Run Quantum Optimization

In [ ]:
print("⚛️ Starting Quantum Route Optimization...\n")

start_time = time.time()
optimized_route = solve_tsp_qubo(drone_base, emergency_locations)
optimization_time = time.time() - start_time

print(f"\n✅ Optimization Complete in {optimization_time:.3f} seconds\n")
print("📍 Optimized Route Sequence:")
for waypoint in optimized_route:
    print(f"   {waypoint['sequence_order']}. {waypoint['id']} → ({waypoint['lat']}, {waypoint['lng']})")

# Calculate metrics
metrics = calculate_route_metrics(optimized_route)
print(f"\n📊 Route Metrics:")
print(f"   Total Distance: {metrics['total_distance_km']} km")
print(f"   Number of Segments: {metrics['segments']}")
print(f"   Average Segment Length: {metrics['avg_segment_km']} km")

## 4️⃣ Visualize the Optimized Route

In [ ]:
plt.figure(figsize=(12, 10))

# Extract coordinates
lats = [wp['lat'] for wp in optimized_route]
lngs = [wp['lng'] for wp in optimized_route]

# Plot the route
plt.plot(lngs, lats, 'b-', linewidth=2, alpha=0.6, label='Optimized Route')

# Plot waypoints
plt.scatter(lngs[0], lats[0], c='green', s=300, marker='s', 
            edgecolors='black', linewidths=2, label='Drone Base', zorder=5)

plt.scatter(lngs[1:], lats[1:], c='red', s=200, marker='o', 
            edgecolors='black', linewidths=2, label='Emergency Sites', zorder=5)

# Add labels
for i, wp in enumerate(optimized_route):
    plt.annotate(f"{i}: {wp['id'].split('_')[-1]}", 
                 (wp['lng'], wp['lat']),
                 xytext=(5, 5), textcoords='offset points',
                 fontsize=9, fontweight='bold')

plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)
plt.title('🚁 PranAIR Quantum-Optimized Delivery Route (Delhi NCR)', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✅ Route visualization complete")

## 5️⃣ Test FastAPI Integration

Now let's test the backend endpoint. Make sure `main.py` is running on port 8000.

In [ ]:
import requests

# Prepare request payload
request_payload = {
    "current_location": {
        "lat": drone_base.lat,
        "lng": drone_base.lng,
        "id": drone_base.id
    },
    "targets": [
        {"lat": loc.lat, "lng": loc.lng, "id": loc.id}
        for loc in emergency_locations
    ]
}

print("📡 Sending route optimization request to FastAPI backend...\n")

try:
    response = requests.post(
        "http://localhost:8000/optimize-route",
        json=request_payload,
        timeout=10
    )
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Backend Response:")
        print(json.dumps(result, indent=2))
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
        
except requests.exceptions.ConnectionError:
    print("⚠️ Could not connect to backend. Make sure main.py is running on port 8000")
except Exception as e:
    print(f"❌ Error: {e}")

## 6️⃣ Performance Comparison: Classical vs Quantum

Compare the quantum-optimized route with a simple sequential (nearest-neighbor) approach.

In [ ]:
# Sequential route (no optimization)
sequential_route = [{"lat": drone_base.lat, "lng": drone_base.lng, "id": drone_base.id}]
for loc in emergency_locations:
    sequential_route.append({"lat": loc.lat, "lng": loc.lng, "id": loc.id})

sequential_metrics = calculate_route_metrics(sequential_route)

print("📊 Performance Comparison\n")
print("=" * 60)
print(f"{'Metric':<30} {'Sequential':<15} {'Quantum'}")
print("=" * 60)
print(f"{'Total Distance (km)':<30} {sequential_metrics['total_distance_km']:<15} {metrics['total_distance_km']}")
print(f"{'Avg Segment (km)':<30} {sequential_metrics['avg_segment_km']:<15} {metrics['avg_segment_km']}")
print(f"{'Waypoints':<30} {sequential_metrics['segments'] + 1:<15} {metrics['segments'] + 1}")
print("=" * 60)

improvement = ((sequential_metrics['total_distance_km'] - metrics['total_distance_km']) / 
               sequential_metrics['total_distance_km'] * 100)
print(f"\n✅ Distance Reduction: {improvement:.1f}%")
print(f"💰 Estimated Fuel Savings: {improvement * 0.8:.1f}%")
print(f"⏱️ Estimated Time Savings: {improvement * 0.6:.1f}%")

## 7️⃣ Future: Real Quantum Hardware (QAOA)

This section demonstrates how to upgrade to real quantum hardware using IBM Quantum or AWS Braket.

**Note**: Currently using classical simulator. To use real quantum:
1. Create IBM Quantum account
2. Get API token
3. Replace `NumPyMinimumEigensolver` with `QAOA`
4. Configure quantum backend

In [ ]:
# Example QAOA implementation (requires IBM Quantum account)
# Uncomment to use with real quantum hardware

"""
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

# Initialize IBM Quantum
service = QiskitRuntimeService(channel="ibm_quantum", token="YOUR_API_TOKEN")
backend = service.backend("ibmq_qasm_simulator")  # Or real quantum hardware

# Configure QAOA
qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA(), reps=3)
qaoa_optimizer = MinimumEigenOptimizer(qaoa)

# Solve using QAOA
result = qaoa_optimizer.solve(qubo)
"""

print("⚛️ QAOA Implementation:")
print("   - Currently using: Classical Simulator (NumPy)")
print("   - Upgrade path: QAOA on IBM Quantum")
print("   - Alternative: D-Wave Quantum Annealer")
print("   - Future: Neutral Atom Quantum Computers")
print("\n✅ For production drone systems, hybrid classical-quantum")
print("   approaches provide the best balance of speed and accuracy.")